In [3]:
import pandas as pd
import re
import csv
import time
import numpy as np
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime
from user_class import User, find_new_friends
import scipy.sparse as sp

//anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [4]:
input_dir = 'paymo_input/'
batch_file = 'batch_payment.csv'
stream_file = 'stream_payment.csv'
batch_path = input_dir + batch_file
stream_path = input_dir + stream_file

test_file = 'batch_payment.csv'
test_path = input_dir + test_file

time: 2.61 ms


In [5]:
batch_dict = {}
batch_dict['time'] = {}
batch_dict['id1'] = {}
batch_dict['id2'] = {}
batch_dict['amount'] = {}
batch_dict['message'] = {}

with open(test_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        batch_dict['time'][i] = row['time']
        batch_dict['id1'][i] = row[' id1']
        batch_dict['id2'][i] = row[' id2']
        batch_dict['amount'][i] = row[' amount']
        batch_dict['message'][i] = row[' message']
        
df_batch = pd.DataFrame.from_dict(batch_dict)

time: 28.2 s


In [7]:
#dictionary columns are randomly ordered - reorder as expected
df_batch = df_batch[['time','id1','id2','amount','message']]
#df_stream = df_stream[['time','id1','id2','amount','message']]

time: 326 ms


In [8]:
givers = df_batch.groupby('id1')
receivers = df_batch.groupby('id2')
partners_1 = {}
partners_2 = {}

time: 1.96 ms


In [9]:
## find all transactions where user <user_id> was giver, then find list of partners in those transactions
for user_id,transactions in givers:
    
    #store list of all transaction partners as list (easiest type to extend later)
    try:
        partners_1[int(user_id)] = list(givers.get_group(user_id)['id2'].astype(int))
   
    #some lines of batch_payment.txt and stream_payment.txt are off - omit malformed entries
    except (KeyError, ValueError) as BadLine:
        print("Skipping invalid key:",user_id)
    
## same as before for all transactions where <user_id> was receiver
for user_id,transactions in receivers:
    
    #store list of all transaction partners as list (easiest type to extend later)
    try: 
        partners_2[int(user_id)] = list(receivers.get_group(user_id)['id1'].astype(int))
        
    #some lines of batch_payment.txt and stream_payment.txt are off - omit malformed entries
    except (KeyError, ValueError) as BadLine:
        print("Skipping invalid key:",user_id)

Skipping invalid key:  no. Even if the union were a matter of economic indifference
Skipping invalid key:  and even if it were to be disadvantageous from the economic standpoint
time: 1min 36s


In [11]:
## it's possible that some users only show up as givers and others only as receivers - combine to master list of all IDs
## in actuality for the provided batch_payment.txt all users show up as givers at least once, but not safe to assume
user_list_1 = np.array(list(partners_1.keys()))
user_list_2 = np.array(list(partners_2.keys()))
user_list = np.unique(np.concatenate([user_list_1,user_list_2]))

time: 23.5 ms


In [12]:
user_master_list = {}

#cycle through all users and agglomerate partners from all transactions
#conversion back and forth between list and numpy array is pretty fast
#lists easier to append to, hence why stored as list, but also wanted to use numpy.unique function.
for user_id in user_list:
    
    pp = []
    
    if user_id in partners_1.keys():
        pp += partners_1[user_id]
        
    if user_id in partners_2.keys():
        pp += partners_2[user_id]
        
    #reduce to (sorted) list of all unique partners
    user_master_list[user_id] = User(user_id, list(np.unique(pp)))

time: 2.32 s


In [ ]:
## use the find_new_friends function (stored in user_class.py) to supplement friend tiers down to level of interest
#creating lists of friends down to 4th-degree connections takes about two minutes for 70,000 users on my macbook pro.
tier_depth = 4

#successively add tiers of friendship to every user in user_master_list
## this takes the longest of any part of the program - about an hour in total.
for tier in range(2,tier_depth+1):
    
    print("Building lists of connections of degree", tier, "for each user...")
    
    for user_id, user_data in user_master_list.items():
        user_data.friends[tier] = find_new_friends(user_master_list,user_data,tier)
            
print("Done. Connections of degree n accessible via User.friends[n]")

Building lists of connections of degree 2 for each user...


In [33]:
## create forward lookup ##
## for each user, dictionary dos links a user id with the level of friendship
for user_id, user_data in user_master_list.items():
    user_data.build_dos()

time: 16.4 s


In [45]:
#first test - have these people had a transaction with each other in the batch data set?
def test1(id1,id2):
    
    #this being python, the following line doesn't take up new memory - just a shorthand
    user_dos = user_master_list[id1].dos
    
    if id2 in user_dos.keys():
        if user_dos[id2] == 1:
            return 'trusted'
    return 'unverified'

#second test - is the transaction partner either a 1st or 2nd degree connection?    
def test2(id1,id2):
    
    #this being python, the following line doesn't take up new memory - just a shorthand
    user_dos = user_master_list[id1].dos
    
    if id2 in user_dos.keys():
        if user_dos[id2] in range (1,3):
            return 'trusted'
    return 'unverified'
    
#third test - is the transaction partner at least a 4th degree connection?    
def test3(id1,id2):

    #this being python, the following line doesn't take up new memory - just a shorthand
    user_dos = user_master_list[id1].dos

    if id2 in user_dos.keys():
        if user_dos[id2] in range(1,5):
            return 'trusted'
    return 'unverified'

time: 10.6 ms


In [46]:
## Now load in second half of data set - this time we choose to flag transactions as verified or unverified.
stream_dict = {}
stream_dict['time'] = {}
stream_dict['id1'] = {}
stream_dict['id2'] = {}
stream_dict['amount'] = {}
stream_dict['message'] = {}

with open(stream_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        stream_dict['time'][i] = row['time']
        stream_dict['id1'][i] = row[' id1']
        stream_dict['id2'][i] = row[' id2']
        stream_dict['amount'][i] = row[' amount']
        stream_dict['message'][i] = row[' message']
        
df_stream = pd.DataFrame.from_dict(stream_dict)

time: 25.2 s


In [51]:
df_stream.head()

,amount,id1,id2,message,time
0,25.32,52575,1120,Spam,2016-11-02 09:49:29
1,19.45,47424,5995,Food for 🌽 😎,2016-11-02 09:49:29
2,14.99,76352,64866,Clothing,2016-11-02 09:49:29
3,13.48,20449,1552,LoveWins,2016-11-02 09:49:29
4,29.94,48676,19395,Jeffs still fat,2016-11-02 09:49:29


time: 8.99 ms


In [59]:
df_stream['id2']

0            1120
1            5995
2           64866
3            1552
4           19395
5           45177
6           16725
7            8306
8           24692
9           66022
10           5637
11           5448
12          59830
13           9013
14           2942
15           3940
16           3197
17           5524
18          13233
19          49196
20          33106
21           2844
22          24068
23          26032
24           3827
25           3081
26          24675
27          21442
28          32471
29          47752
            ...  
2993519     48063
2993520      7015
2993521      5199
2993522     10096
2993523      9298
2993524     11303
2993525      3832
2993526     13925
2993527     33400
2993528     32430
2993529      3031
2993530        14
2993531     26160
2993532     22694
2993533      2908
2993534      6256
2993535     80358
2993536     17053
2993537     26631
2993538        59
2993539     73642
2993540     11765
2993541     38399
2993542      4282
2993543   

time: 929 ms


In [55]:
## create output columns
#df_stream['test1'] = df_stream.apply(lambda x: test1(int(x['id1']), int(x['id2'])), axis=1)
#df_stream['test2'] = pd.Series('unverified', index=df_stream.index)
#df_stream['test3'] = pd.Series('unverified', index=df_stream.index)
df_stream['test4'] = df_stream.apply(lambda x: int(x['id2']))



IndexError: ('id2', 'occurred at index amount')

time: 22.8 ms


In [44]:
test3(5,1)

False

time: 1.94 ms


In [10]:
## new approach - try to create a connectivity matrix - then, can figure out second-tier neighbors and such by multiplying matrix
# by itself
## conveniently, the list of users already ranges from 0 to 77,359 without gaps
## therefore, can use usernumbers directly as row/column indices in connectivity matrix

## takes about two minutes on 2016 macbook pro with 16GB of ram to convert to sparse matrix.

n_users = len(user_master_list.keys())
connectivity = np.zeros([n_users,n_users])

for user_id, user_data in user_master_list.items():
    for ff in user_data.friends[1]:
        #connections are bi-directional, so we fill in two spots in the connectivity matrix.
        connectivity[user_id,ff] = 1
        connectivity[ff,user_id] = 1
        
#multiplying 100k by 100k numpy matrices too taxing - try sparse matrices.
csp = sp.csr_matrix(connectivity)

time: 1min 50s


In [ ]:
#credit to Philipp Singer for his excellent tutorial on how to save the output of big matrix
#multiplication to disk: http://www.philippsinger.info/?p=464
l = n_users
 
f = tb.open_file('product_2.h5', 'w')
filters = tb.Filters(complevel=5, complib='blosc')
out = f.create_carray(f.root, 'data', tb.Float32Atom(), shape=(l, l), filters=filters)
 
bl = 1000 #this is the number of rows we calculate each loop
b = csp.tocsc() #we slice b on columns, csc improves performance
 
#slice by row
for i in range(0, l, bl):
    out[:,i:min(i+bl, l)] = (csp.dot(b[:,i:min(i+bl, l)])).toarray()
 
f.close()

In [17]:
h5 = tb.open_file('product_2.h5', 'r')
a = h5.root.data

## the advantage of this system is that only one row gets loaded into memory at a time -
# although we are saving a list of second-tier connections
for user_id in range(10):
    row = a[user_id,:] 
    user_master_list[user_id].friends[2] = list(np.nonzero(row))
    print(len(user_master_list[user_id].friends[2]))
    time.sleep(2)
    
h5.close()

(array([   0,    1,    2, ..., 4997, 4998, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(array([   0,    1,    2, ..., 4993, 4997, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(array([   0,    1,    2, ..., 4993, 4998, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(array([   0,    1,    2, ..., 4996, 4997, 4999]),)
(77360,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


KeyboardInterrupt: 

time: 7.11 s


In [27]:
h5.close()

time: 737 µs


In [6]:
## BIG PROBLEM - SKIPPED OVER A BUNCH OF ENTRIES
batch_dict['message'][377592]
#print(len(batch_dict['message'].keys()))

' 🇨🇴🇨🇴🇨🇴🇨🇴👍🏼🎉 '